# Sandbox to play around with data

In [1]:
import qgrid
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import socket as socket
import os as os
import sys as sys
import multiprocessing as mp
import itertools as it
import h5py

### hapROH imports
#sys.path.insert(0,"/n/groups/reich/hringbauer/git/HAPSBURG/package/")  # hack to get development package first in path
sys.path.insert(0,"/n/groups/reich/hringbauer/git/hapBLOCK/package/hapBLOCK/IO/")  # hack to get development package first in path
from h5_load import get_coverage,get_genos_pairs,get_idx_iid,get_markers_good

socket_name = socket.gethostname()
print(socket_name)

if socket_name.startswith("compute-"):
    print("HSM O2 Computational partition detected.")
    path = "/n/groups/reich/hringbauer/git/hapBLOCK/"  # The Path on Harvard Cluster
else:
    raise RuntimeWarning("No compatible machine. Check!!")

os.chdir(path)  # Set the right Path (in line with Atom default)
# Show the current working directory. Should be HAPSBURG/Notebooks/ParallelRuns

print(os.getcwd())
print(f"CPU Count: {mp.cpu_count()}")

sys.path.insert(0, path)

compute-e-16-230.o2.rc.hms.harvard.edu
HSM O2 Computational partition detected.
/n/groups/reich/hringbauer/git/hapBLOCK
CPU Count: 28


In [2]:
f = h5py.File("./data/hdf5/1240k_v43/ch20.h5", "r") # Load for Sanity Check. See below!

In [3]:
list(f)

['calldata', 'samples', 'variants']

In [39]:
f["calldata/HS"][10001,500]

1048575

In [27]:
f["calldata/AD"][10004,500,:]

array([18, 21, -1, -1], dtype=int16)

In [30]:
np.shape(f["calldata/HS"])

(15571, 14523)

# Find distant relatives

In [64]:
df = pd.read_csv("/n/groups/reich/DAVID/V44/V44.0/v44.0.anno", sep="\t", low_memory=False)
col_fam = "Family ID and position within family"
col_cov = "Coverage on autosomal targets"
col_iid = "Master ID"
col_iid1 = "Version ID"
col_age = 'Average of 95.4% date range in calBP (defined as 1950 CE)  '
df[col_cov]= pd.to_numeric(df[col_cov], errors="coerce")

In [34]:
df3 = df[df[col_fam].str.contains("2nd")]
df3 = df3.sort_values(by=col_cov, ascending=False)

In [37]:
df3[[col_fam, col_cov]]

,Family ID and position within family,Coverage on autosomal targets
9179,"Russia, Novosibirsk Oblast, Toguchinskiy Rayon...",30.095700
9180,"Russia, Novosibirsk Oblast, Toguchinskiy Rayon...",30.095700
9375,"Russia, Lower Tyumechin 1 Family A (2 members)...",28.435000
9374,"Russia, Lower Tyumechin 1 Family A (2 members)...",28.435000
9398,"Germany, Irlbach LKR Family A (4 members) (I58...",27.786000
...,...,...
15127,Germany cross-site Family A (11 members) (At K...,0.001349
15146,"Germany, Oberottmarshausen-Kiesgrube Lauter Fa...",0.001194
15166,"Germany, Haunstetten-Postillionstraße Family A...",0.001181
16138,"Slovakia, Bohemia, Ivanka pri Dunaji Family A ...",NaN


In [68]:
df3[col_fam].values[9]

'Russia, Altai Mountains, Yenisey River, left bank of Karasuk tributary, Karasuk III Family A (5 members) I3388-13950-I3949-I7714 have a mother, father, son1 and son2 relationsjip in that order. I3951 is a first deg relation to I13950 and a 2nd or 3rd to I3949 and I16714 '

In [67]:
df3[df3[col_iid].str.contains("I2138|I2139")][[col_iid1, col_iid, col_age, col_cov]]

,Version ID,Master ID,Average of 95.4% date range in calBP (defined as 1950 CE),Coverage on autosomal targets
9179,I2139.DG,I2139,8897.0,30.095700
9180,I2139.SG,I2139,8897.0,30.095700
7704,I2139,I2139,8897.0,8.797203
7703,I2138,I2138,8793.0,8.338042


In [66]:
df3[df3[col_iid].str.contains("I2168|I6801")][[col_iid1, col_iid, col_age, col_cov]]

,Version ID,Master ID,Average of 95.4% date range in calBP (defined as 1950 CE),Coverage on autosomal targets
18084,I6801_new.SG,I6801,4012.0,26.675296
9443,I6801.SG,I6801,4012.0,6.236590
9181,I2168.SG,I2168,4078.0,5.581440
107,I6801,I6801,4012.0,4.073639
1602,I2168,I2168,4078.0,3.998000


In [71]:
df3[df3[col_iid].str.contains("I3951|I3949")][[col_iid1, col_iid, col_age, col_cov]]

,Version ID,Master ID,Average of 95.4% date range in calBP (defined as 1950 CE),Coverage on autosomal targets
12970,I3949.DG,I3949,4618.0,25.316534
12971,I3949.SG,I3949,4618.0,25.316534
9251,I3951.SG,I3951,4709.0,5.854190
1855,I3949,I3949,4618.0,3.928000
1818,I3951,I3951,4709.0,3.742000
